In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle

In [23]:
data = pd.read_csv('Churn_Modelling.csv') 
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [24]:
 # Preprocess the data - drop irrelevant data
data = data.drop(['RowNumber','CustomerId','Surname' ],axis=1)

In [25]:
# do the same things as we did in classification problem
# Encoding for Categorical column - gender, country

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender .fit_transform(data['Gender'])

In [26]:
# one hot encoding on geography

one_hot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()

geo_encoded_df = pd.DataFrame(geo_encoded , columns= one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [27]:
# Combine the data
data = pd.concat([data.drop(['Geography'],axis=1), geo_encoded_df],axis=1)

In [28]:
# Divide the dataset into independent and dependent dataset

X = data.drop('Exited', axis=1)
y = data['Exited']

# Split the data

X_train, X_test , y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Scale down the feature
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
# Save the encoders and scalers

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geo.pkl','wb')as file:
    pickle.dump(one_hot_encoder_geo, file)

with open ('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

ANN Problem Statement

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [33]:
# Build the model
model = Sequential([
    Dense(64,activation='relu', input_shape = (X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)    # output layer regression
    # If I dont write activation function then linear acitvation function is applied
])

    # Compile the model
model.compile(optimizer='adam', loss = 'mean_absolute_error', metrics=['mae'])

model.summary()

/Users/rishabh/ML/Projects/Churn Prediction Using ANN/avenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
# Setup the tensorboard

log_dir = "regression_logs/fit"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq= 1)

In [38]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [39]:
history = model.fit(
    X_train,y_train,
    validation_data =(X_test, y_test), 
    epochs = 100,
    callbacks = [tensorboard_callback, early_stopping_callback]
)

Epoch 1/100


2026-02-28 15:57:50.145553: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.2408 - mae: 0.2408 - val_loss: 0.2088 - val_mae: 0.2088
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2232 - mae: 0.2232 - val_loss: 0.2159 - val_mae: 0.2159
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.2258 - mae: 0.2258 - val_loss: 0.2352 - val_mae: 0.2352
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2327 - mae: 0.2327 - val_loss: 0.2331 - val_mae: 0.2331
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2465 - mae: 0.2465 - val_loss: 0.2590 - val_mae: 0.2590
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2551 - mae: 0.2551 - val_loss: 0.2407 - val_mae: 0.2407


In [42]:
 %load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [45]:
%tensorboard --logdir regression_logs/fit

Reusing TensorBoard on port 6008 (pid 56705), started 0:00:39 ago. (Use '!kill 56705' to kill it.)

In [46]:
# Evaluate the model on test data

test_loss, test_mae = model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2088 - mae: 0.2088
Test MAE : 0.20877498388290405


In [47]:
model.save('regression_model.h5')